In [4]:
import csv
import time
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, clear_output
import pandas as pd

### Cargar Data

In [5]:
def load_data(csv_file_path):
    # Cargar los datos del archivo CSV en un DataFrame de Pandas
    df = pd.read_csv(csv_file_path)
    return df

def plot_data_by_id_ubi(df):
    # Crear una figura con Plotly Express para visualizar los datos por id_ubi
    fig = px.line(df, x='Timestamp', y=['CO2', 'Temperature', 'Humidity'], color='id_ubi')

    # Actualizar las opciones de diseño
    fig.update_layout(
        title='Sensor Data by id_ubi',
        xaxis=dict(title='Time'),
        yaxis=dict(title='Value'),
    )

    # Mostrar el gráfico
    fig.show()

csv_file_path = "results.csv"  # Ruta del archivo CSV

# Cargar los datos del archivo CSV
data = load_data(csv_file_path)

# Graficar los datos por id_ubi
plot_data_by_id_ubi(data)


### Enviar Notificaciones

In [13]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from getpass import getpass
from email.mime.image import MIMEImage
import smtplib
import io
import matplotlib.pyplot as plt
import base64

In [20]:
def analyze_air_quality(df):
    # Calcular promedios por ubicación
    location_averages = df.groupby('id_ubi').agg({
        'CO2': 'mean',
        'Temperature': 'mean',
        'Humidity': 'mean'
    }).round(2)
    
    # Ordenar ubicaciones por nivel de CO2 (ascendente = mejor calidad de aire)
    location_ranking = location_averages.sort_values('CO2')
    
    return location_ranking

In [36]:
def generate_plot(df):
    # Crear figura con subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
    
    # Graficar CO2 para cada ubicación
    for location in df['id_ubi'].unique():
        location_data = df[df['id_ubi'] == location]
        ax1.plot(location_data['Timestamp'], location_data['CO2'], label=f'Ubicación {location}')
    ax1.set_ylabel('CO2 (ppm)')
    ax1.set_title('Niveles de CO2 por Ubicación')
    ax1.legend()
    ax1.grid(True)

    # Graficar Temperatura
    for location in df['id_ubi'].unique():
        location_data = df[df['id_ubi'] == location]
        ax2.plot(location_data['Timestamp'], location_data['Temperature'], label=f'Ubicación {location}')
    ax2.set_ylabel('Temperatura (°C)')
    ax2.set_title('Temperatura por Ubicación')
    ax2.grid(True)

    # Graficar Humedad
    for location in df['id_ubi'].unique():
        location_data = df[df['id_ubi'] == location]
        ax3.plot(location_data['Timestamp'], location_data['Humidity'], label=f'Ubicación {location}')
    ax3.set_ylabel('Humedad (%)')
    ax3.set_title('Humedad por Ubicación')
    ax3.grid(True)

    plt.tight_layout()
    
    # Guardar imagen en memoria
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png', dpi=300, bbox_inches='tight')
    img_buffer.seek(0)
    plt.close()
    
    return img_buffer

In [37]:
def generate_air_quality_html(df, location_ranking):
    # Generar HTML con estilos modernos
    html = f"""
    <html>
        <head>
            <style>
                body {{
                    font-family: 'Arial', sans-serif;
                    line-height: 1.6;
                    margin: 0;
                    padding: 20px;
                    background-color: #f5f5f5;
                }}
                .container {{
                    max-width: 800px;
                    margin: 0 auto;
                    background-color: white;
                    padding: 20px;
                    border-radius: 10px;
                    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                }}
                h1, h2 {{
                    color: #2c3e50;
                    border-bottom: 2px solid #3498db;
                    padding-bottom: 10px;
                }}
                .ranking-card {{
                    background-color: #fff;
                    padding: 15px;
                    margin: 10px 0;
                    border-radius: 5px;
                    border-left: 5px solid;
                }}
                .best {{
                    border-left-color: #2ecc71;
                }}
                .warning {{
                    border-left-color: #e74c3c;
                    background-color: #fadbd8;
                }}
                .stats {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                    gap: 15px;
                    margin: 20px 0;
                }}
                .stat-card {{
                    background-color: #fff;
                    padding: 15px;
                    border-radius: 5px;
                    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                }}
                img {{
                    max-width: 100%;
                    height: auto;
                    margin: 20px 0;
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Reporte de Calidad del Aire</h1>
                
                <div class="ranking-card best">
                    <h2>Mejor Ubicación</h2>
                    <p>La ubicación con mejor calidad de aire es: <strong>Ubicación {location_ranking.index[0]}</strong></p>
                    <p>CO2 Promedio: {location_ranking.iloc[0]['CO2']} ppm</p>
                </div>

                <div class="ranking-card warning">
                    <h2>⚠️ Alerta de Calidad de Aire</h2>
                    <p>La ubicación con niveles más altos de CO2 es: <strong>Ubicación {location_ranking.index[-1]}</strong></p>
                    <p>CO2 Promedio: {location_ranking.iloc[-1]['CO2']} ppm</p>
                    <p>Se recomienda mejorar la ventilación en esta área.</p>
                </div>

                <h2>Top 5 Ubicaciones (Ordenadas por Calidad de Aire)</h2>
                <div class="stats">
    """
    
    # Agregar cada ubicación al HTML
    for idx, (location, row) in enumerate(location_ranking.iterrows(), 1):
        color = "#2ecc71" if idx == 1 else "#e74c3c" if idx == len(location_ranking) else "#3498db"
        html += f"""
                    <div class="stat-card" style="border-left: 5px solid {color}">
                        <h3>#{idx} - Ubicación {location}</h3>
                        <p>CO2: {row['CO2']} ppm</p>
                        <p>Temperatura: {row['Temperature']}°C</p>
                        <p>Humedad: {row['Humidity']}%</p>
                    </div>
        """

    # Agregar referencia a la imagen adjunta y cerrar el HTML
    html += """
                </div>
                
                <h2>Gráfica de Tendencias</h2>
                <img src="cid:graficas_tendencias" alt="Gráficas de tendencias">
            </div>
        </body>
    </html>
    """
    
    return html

In [38]:
def enviar_correo_gmail(remitente, password, destinatario, asunto, texto, html, imagen_buffer):
    # Crear mensaje
    mensaje = MIMEMultipart('related')
    mensaje['Subject'] = asunto
    mensaje['From'] = remitente
    mensaje['To'] = destinatario
    
    # Crear la parte alternativa (texto plano y HTML)
    msgAlternative = MIMEMultipart('alternative')
    mensaje.attach(msgAlternative)
    
    # Agregar texto plano
    parte_texto = MIMEText(texto, 'plain')
    msgAlternative.attach(parte_texto)
    
    # Agregar HTML
    parte_html = MIMEText(html, 'html')
    msgAlternative.attach(parte_html)
    
    # Adjuntar la imagen y establecer el Content-ID
    imagen = MIMEImage(imagen_buffer.getvalue())
    imagen.add_header('Content-ID', '<graficas_tendencias>')
    imagen.add_header('Content-Disposition', 'inline', filename='graficas_tendencias.png')
    mensaje.attach(imagen)
    
    # Conectar y enviar
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as servidor:
            servidor.login(remitente, password)
            servidor.sendmail(remitente, destinatario, mensaje.as_string())
        print("Correo enviado exitosamente")
    except Exception as e:
        print(f"Error al enviar correo: {str(e)}")

In [39]:
    # Configuración del correo
remitente = 'alumnoportal123@gmail.com'
password = 'ksyevfuheiwgkqmw'
destinatario = 'marlon.durand@unmsm.edu.pe'
    
    # Cargar y analizar datos
csv_file_path = "results.csv"
df = pd.read_csv(csv_file_path)

    # Analizar datos
location_ranking = analyze_air_quality(df)
    
    # Generar gráfica
plot_buffer = generate_plot(df)
    
    # Generar contenido del correo
asunto = "Reporte de Calidad del Aire - Análisis Diario"
texto = """
    Reporte de Calidad del Aire
    
    Este es un resumen automático de las mediciones de calidad del aire.
    Por favor, visualice este correo en HTML para ver el reporte completo con gráficas.
    """
    
html = generate_air_quality_html(df, location_ranking)
    
    # Enviar correo
enviar_correo_gmail(remitente, password, destinatario, asunto, texto, html, plot_buffer)

Correo enviado exitosamente
